# Imports

In [1]:
import gc
gc.enable()

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import scipy.stats as st
import pandas as pd
pd.set_option('precision', 4)
pd.set_option('display.max_columns', None)

In [2]:
# %%capture
# !pip install scikit-learn==1.0.2
# !pip install scikit-learn-intelex

# from sklearnex import patch_sklearn
# patch_sklearn()

In [3]:
train = pd.read_csv('../input/netflix-appetency/train.csv')
test = pd.read_csv('../input/netflix-appetency/test.csv')

# Data Cleaning

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Columns: 509 entries, id to feature_506
dtypes: float64(143), int64(274), object(92)
memory usage: 271.8+ MB


In [5]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Columns: 508 entries, id to feature_506
dtypes: float64(143), int64(273), object(92)
memory usage: 116.3+ MB


In [6]:
train_clean = train.copy()
test_clean = test.copy()
test_clean.drop(['id'], axis=1, inplace=True)

### Removing empty rows and columns

In [7]:
train_clean.dropna(how='all', axis=0, inplace=True) #rows with all values missing

Cannot drop rows in test set. 

In [8]:
train_clean.dropna(how='all', axis=1, inplace=True) #columns with all values missing
test_clean.dropna(how='all', axis=1, inplace=True)

### Removing duplicate rows

In [9]:
features = list(test_clean.columns)

In [10]:
train_clean.duplicated(subset=features).sum(), test_clean.duplicated(subset=features).sum()

(1, 0)

In [11]:
train_clean.drop_duplicates(subset=features, keep='first', ignore_index=True, inplace=True)

### Dropping zero-variance features (only one unique value in a column)

In [12]:
zero_var_cols = list(filter(lambda col: train[col].nunique() == 1, features))

In [13]:
train_clean.drop(zero_var_cols, axis=1, inplace=True)
test_clean.drop(zero_var_cols, axis=1, inplace=True)

### Dropping features with a majority of missing values

In [14]:
features = list(test_clean.columns)

In [15]:
def get_majority_missing_cols(df):
    return list(filter(lambda col: df[col].isna().sum() / len(df) >= 0.5, features))

In [16]:
majority_missing_train = get_majority_missing_cols(train_clean)
majority_missing_test = get_majority_missing_cols(test_clean)

In [17]:
set(majority_missing_test) - set(majority_missing_train), set(majority_missing_train) - set(majority_missing_test)

(set(), set())

Same columns have majority missing values in both datasets.

In [18]:
train_clean.drop(labels=majority_missing_train, axis=1, inplace=True)
test_clean.drop(labels=majority_missing_train, axis=1, inplace=True)

### Handling datetime columns

In [19]:
features = list(test_clean.columns)

In [20]:
train_clean.select_dtypes(include='object').head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_41,feature_42,feature_54,feature_58,feature_97,feature_133,feature_134,feature_135,feature_152,feature_163,feature_164,feature_165,feature_185,feature_186,feature_189,feature_191,feature_192,feature_194,feature_199,feature_200,feature_201,feature_202,feature_204,feature_205,feature_239,feature_240,feature_241,feature_242,feature_243,feature_244,feature_246,feature_247,feature_250,feature_251,feature_270,feature_271,feature_272,feature_273,feature_274,feature_275,feature_276,feature_277,feature_278,feature_279,feature_280,feature_281,feature_282,feature_283,feature_284,feature_285,feature_286,feature_287,feature_288,feature_289,feature_290,feature_291,feature_292,feature_293,feature_336,feature_337,feature_338,feature_371
0,C0,C0,C1,C5,C11,C0,C60,C17795,C413,C3620,C0,C0,C0,C19,C0,C0,C0,C0,C0,C0,C0,C0,C0,C0,C413,C4,C41,C16133,C2,C16,C0,C1,C1,C2,C1483,C6,C20,11/10/2017,11/10/2017,09/11/2017,11/10/2017,NaN,11/10/2017,NaN,NaN,C5,C0,C1,C1,C1,C0,C2,C140,C91,C1,C0,C0,C66,C63,C25,C10,C2,C1,C0,C0,C105,C86,C30,C13,C2,C1,C0,C2,C166,C130,C46,C22,C10,C7,C6,C0,C0,C0,C0
1,C0,C0,C3,C5,C1,C2,C50,C14050,C413,C2819,C0,C0,C0,C23,C0,C0,C0,C0,C0,C0,C0,C0,C0,C0,C413,C7,C41,C21195,C2,C16,C0,C3,C1,C11,C1483,C8,C21,25/03/2015,25/03/2015,NaN,02/04/2017,25/03/2015,25/03/2015,NaN,25/03/2015,C9,C0,C2,C2,C1,C14,C3,C193,C66,C1,C0,C0,C80,C74,C29,C13,C5,C2,C1,C0,C128,C104,C36,C18,C6,C3,C2,C0,C130,C105,C36,C18,C6,C3,C2,C0,C0,C0,C0
2,C0,C0,C3,C5,C2,C1,C76,C23471,C413,C4794,C0,C0,C0,C22,C0,C0,C0,C0,C0,C0,C0,C0,C0,C0,C413,C7,C41,C11390,C2,C16,C0,C3,C1,C11,C1483,C4,C11,27/01/2015,27/01/2015,13/05/2017,24/03/2016,27/01/2015,27/01/2015,NaN,27/01/2015,C9,C0,C2,C2,C1,C14,C3,C175,C66,C1,C0,C0,C80,C74,C29,C13,C5,C2,C1,C0,C128,C104,C36,C18,C6,C3,C2,C0,C130,C105,C36,C18,C6,C3,C2,C0,C0,C0,C0
3,C0,C0,C1,C5,C1,C0,C58,C16929,C413,C3344,C0,C0,C0,C2,C0,C0,C0,C0,C0,C0,C0,C0,C0,C0,C413,C7,C41,C22367,C2,C16,C0,C3,C1,C1,C1483,C2,C5,05/02/2014,01/04/2014,NaN,01/04/2014,01/04/2014,01/04/2014,31/03/2014,05/02/2014,C0,C0,C2,C2,C1,C0,C3,C193,C13,C1,C0,C0,C78,C73,C28,C12,C3,C1,C0,C0,C103,C84,C28,C13,C2,C1,C0,C0,C104,C83,C28,C13,C2,C1,C0,C0,C0,C0,C0
4,C0,C0,C3,C3,C11,C2,C48,C13941,C413,C2753,C0,C0,C0,C3,C1,C1,C1,C1,C1,C1,C1,C0,C0,C0,C413,C5,C1,C1652,C2,C16,C0,C0,C1,C11,C1483,C12,C11,20/07/2009,10/09/2013,14/02/2017,10/09/2013,24/01/2016,10/09/2013,24/01/2016,23/03/2011,C2,C0,C1,C1,C1,C0,C2,C60,C31,C1,C0,C0,C64,C61,C23,C10,C2,C1,C0,C0,C86,C69,C24,C11,C2,C1,C0,C0,C114,C93,C24,C11,C2,C1,C0,C0,C0,C0,C1


Datetime columns: feature_191, 192, 194, 199, 200, 201, 202, 204

In [21]:
date_features = ['feature_191', 'feature_192', 'feature_194', 'feature_199',
                 'feature_200', 'feature_201', 'feature_202', 'feature_204']

In [22]:
get_day = lambda val: -1 if str(val) == 'nan' else int(str(val).strip().split('/')[0])

get_month = lambda val: -1 if str(val) == 'nan' else int(str(val).strip().split('/')[1])

get_year = lambda val: -1 if str(val) == 'nan' else int(str(val).strip().split('/')[2])

In [23]:
for col in date_features:
    train_clean[col+'_day'] = train_clean[col].apply(get_day)
    train_clean[col+'_month'] = train_clean[col].apply(get_month)
    train_clean[col+'_year'] = train_clean[col].apply(get_year)
    
    test_clean[col+'_day'] = test_clean[col].apply(get_day)
    test_clean[col+'_month'] = test_clean[col].apply(get_month)
    test_clean[col+'_year'] = test_clean[col].apply(get_year)

In [24]:
train_clean.drop(date_features, axis=1, inplace=True)
test_clean.drop(date_features, axis=1, inplace=True)

In [38]:
date_features = [f for f in test_clean.columns if f.endswith(('_day', '_month'))]
#excluding '_year' columns since we will be treating them as numerical features

### Removing categorical columns with skewed proportions
1. most frequent category > 95%
2. most frequent category > 80% and all other categories <= 1% (many insignificant categories) 

In [26]:
cat_features = list(train_clean.select_dtypes(include='object').columns)

In [27]:
skewed_cat = []
for col in cat_features:
    most_frequent = train_clean[col].value_counts(normalize=True).sort_values()[-1]
    second_most_frequent = train_clean[col].value_counts(normalize=True).sort_values()[-2]
    if most_frequent > 0.95 or (most_frequent > 0.8 and second_most_frequent <= 0.01):
        skewed_cat.append(col)

In [28]:
train_clean.drop(skewed_cat, axis=1,inplace=True)
test_clean.drop(skewed_cat, axis=1,inplace=True)

### Encoding categorical columns

In [29]:
cat_features = list(train_clean.select_dtypes(include='object').columns)

In [30]:
cat_encode = lambda val: -1 if str(val) == 'nan' else int(str(val)[1:])

train_clean[cat_features] = train_clean[cat_features].applymap(cat_encode)
test_clean[cat_features] = test_clean[cat_features].applymap(cat_encode)

In [31]:
train_clean[cat_features].isna().sum().sum(), test_clean[cat_features].isna().sum().sum()

(0, 0)

No missing values in categorical columns.

### Handling missing values in numerical features

* Categorical columns have no missing values.  
* Date columns created by us already have missing values filled with -1.  
* Columns with a majority of missing values have already been dropped.  
* For the remaining numerical features, we will fill with median values for each column.

In [32]:
num_features = [f for f in test_clean.columns 
                if (f not in cat_features and f not in date_features)]

In [34]:
for col in num_features:
    train_clean[col].fillna(train_clean[col].median(), inplace=True)
    test_clean[col].fillna(train_clean[col].median(), inplace=True)

# Processed dataset

In [39]:
#storing as type category to identify categorical features. 

train_clean[cat_features] = train_clean[cat_features].astype('category')
test_clean[cat_features] = test_clean[cat_features].astype('category')

train_clean[date_features] = train_clean[date_features].astype('category')
test_clean[date_features] = test_clean[date_features].astype('category')

In [40]:
train_clean.to_csv('train_clean.csv', index=False)
test_clean.to_csv('test_clean.csv', index=False)